In [1]:
###########################################################
#                                                         #
#               TRAITEMENT SIMUS FORTRAN                  #
#                     STATISTIQUES                        #
#                                                         #
#                         avec                            #
#                   - lecture arborescence                #
#                  analyse                                #
#                   - Delta Fluo                          #
#                   - Delta Energy GMol                   #
#                   - R/L Ca+                             #
#                   - distance inter ionique              #
#                   - trajectoire GMol                    #
#                                                         #
#                          02/2020     Adrien Poindron    #
###########################################################

In [9]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
matplotlib.rcParams['figure.figsize'] = [10.5,7]
matplotlib.rcParams.update({'font.size': 25})

# Imports

In [3]:
# %pylab

import numpy
import matplotlib
from matplotlib.patches import Ellipse
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

matplotlib.rcParams.update({'font.size': 21})
# matplotlib.rcParams["figure.dpi"] = 300

from IPython.display import display
from IPython.core.pylabtools import figsize, getfigs

from pylab import *
from numpy import *

from scipy.optimize import curve_fit

In [4]:
import os
from os import listdir
from os.path import isfile, join

import tkinter as tk
from tkinter import filedialog
%matplotlib qt

import matplotlib.patches as patches

from pathlib import Path
from matplotlib import gridspec
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

In [5]:
from natsort import natsorted   # natural sort marche de ouf
import re                       # find '/' in strings
import timeit
cm = pylab.get_cmap('summer')   # colormap par défaut color=cm(k/len(array))

In [609]:
# Fonctions persos de Jofre et Adrien

# %run function_jofre.ipy
%run "../../Functions/1.2/data_fct_Adrien_point_by_point.py"

In [8]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol
wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


# Sélection fichier

In [10]:
## GUI for data loading
# Select one data file all the way down to the directories
# Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+00V_D1.0_S1.0RFG.dat

file_cfg, slash_cfg, all_subdir = load_gui(filter_nocomplete=1)

file_path  = file_cfg[0]
dir_path   = file_cfg[1]
work_rep   = file_cfg[2]
filename   = file_cfg[3]

myslashpos = slash_cfg[0]
slashcond  = slash_cfg[1]

> Répertoire : /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC01_RF08/Try00
> Filename : Temp_SimuType0_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG.dat
> myslashpos | [0, 5, 12, 22, 33, 47, 56, 66]
> slashcond | 6
Points deleted because they were not complete []   0 pt(s)
Total number of data directories 962


In [11]:
## Condition for data
points_and_coord, condition_parameters = simu_conditions(all_subdir, myslashpos, slashcond, filename)

> condition names ['DC', 'RF']
> number of points 74
000 > DC01_RF08
001 > DC01_RF09
002 > DC01_RF10
003 > DC01_RF11
004 > DC01_RF14
005 > DC01_RF15
006 > DC01_RF16
007 > DC01_RF17
008 > DC01_RF18
009 > DC01_RF19
010 > DC01_RF20
011 > DC03_RF08
012 > DC03_RF09
013 > DC03_RF10
014 > DC03_RF11
015 > DC03_RF14
016 > DC03_RF15
017 > DC03_RF16
018 > DC03_RF17
019 > DC03_RF18
020 > DC03_RF19
021 > DC03_RF20
022 > DC05_RF08
023 > DC05_RF09
024 > DC05_RF10
025 > DC05_RF11
026 > DC05_RF14
027 > DC05_RF15
028 > DC05_RF16
029 > DC05_RF17
030 > DC05_RF18
031 > DC07_RF08
032 > DC07_RF09
033 > DC07_RF10
034 > DC07_RF11
035 > DC07_RF14
036 > DC07_RF15
037 > DC07_RF16
038 > DC07_RF17
039 > DC07_RF18
040 > DC09_RF08
041 > DC09_RF09
042 > DC09_RF10
043 > DC09_RF11
044 > DC09_RF14
045 > DC09_RF15
046 > DC09_RF16
047 > DC09_RF17
048 > DC09_RF18
049 > DC10_RF08
050 > DC10_RF09
051 > DC10_RF10
052 > DC10_RF11
053 > DC10_RF14
054 > DC10_RF15
055 > DC10_RF16
056 > DC10_RF17
057 > DC10_RF18
058 > DC11_RF08
059

In [ ]:
## Data loading (1D arrays)
data_name, num_runs, PMandT, Gmol_data, r_LC_clip, dim_nu \
= data_retrieve(all_subdir, points_and_coord, condition_parameters, slash_cfg) 

data0 = data_name[0]
data2 = data_name[1]
data4 = data_name[2]

PMvar = PMandT[0]
Tvar = PMandT[1]

deltaEc = Gmol_data[0]
deltaEcRel = Gmol_data[1]
t_c = Gmol_data[2]

# Analyser un seul point et tous ses essais

In [505]:
def load_condition(all_subdir, cond_name, myslashpos, slashcond):
    
    pt_to_plot = [i for i in all_subdir if cond_name in i]
    address = pt_to_plot[0]
    onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and not "xva" in f and ".dat" in f]
    fluo_var, GMol_var, cloud_atlas, t_aux, r_LC_clip = load_T_PM_cloud_GMol(all_subdir[0],flag_plot=0)
    tt_Inje, T_CM_Inje, T_aux_Inje, PM_Inje = plot_T_and_PM_Inje(address,sort(onlyfiles)[0].strip('.dat'))


    shapevar = (len(pt_to_plot),len(cloud_atlas[2]))
    PMvar = np.zeros(len(pt_to_plot))
    Tvar = np.zeros(len(pt_to_plot))
    SNR = np.zeros(len(pt_to_plot))
    tt = np.zeros(shapevar)
    T_aux = np.zeros((len(pt_to_plot),len(cloud_atlas[2]),3))
    T_CM = np.zeros((len(pt_to_plot),len(cloud_atlas[2]),3))
    PM = np.zeros(shapevar)
    
    shapevar = (len(pt_to_plot),len(tt_Inje))
    tt_Inje    = np.zeros(shapevar)
    T_CM_Inje  = np.zeros((len(pt_to_plot),62,3))
    T_aux_Inje = np.zeros((len(pt_to_plot),62,3))
    PM_Inje    = np.zeros(shapevar)

    for k, address in enumerate(pt_to_plot):
        fluo_var, GMol_var, cloud_atlas, t_aux, r_LC_clip = load_T_PM_cloud_GMol(address,flag_plot=0)
        PMvar[k] = fluo_var[0] ; Tvar[k] = fluo_var[1] ; SNR[k] = fluo_var[2]
        tt[k] = cloud_atlas[0] ; T_aux[k] = cloud_atlas[1] ; T_CM[k] = cloud_atlas[2] ; PM[k] = cloud_atlas[3]
        
        aa, bb, cc, dd = plot_T_and_PM_Inje(address,sort(onlyfiles)[0].strip('.dat'))
        tt_Inje[k] = aa;
        T_CM_Inje[k] = bb;
        T_aux_Inje[k] = cc;
        PM_Inje[k] = dd
        
    fluo_signal = [PM, SNR, PMvar]
    temp_signal = [T_aux, T_CM, Tvar]
    Inje_signal = [tt_Inje, T_CM_Inje, T_aux_Inje, PM_Inje]
        
    return fluo_signal, temp_signal, Inje_signal, tt

In [731]:
# load one condition
# plot temperature during SimuType2
# compare to full plot SimuType all

my_cond = 'DC05_RF09'
pt_to_plot = [i for i in all_subdir if my_cond in i]
my_try = 1
address = pt_to_plot[my_try]

# Check with all the Temp curves from one try
onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and not "xva" in f and ".dat" in f]
temps, temperature, fluo = plot_T_and_PM_Init_Inje_Evol(address+'/',sort(onlyfiles)[0].strip('.dat'),1,'test all simu')

# Cloud size
r_LC_clip, dim_nu = cloud_size(address,onlyfiles)
str_load = str(address+'/trj'+sort(onlyfiles)[2][4:]).strip('.dat')
t_GMol,_, _, r_z_GMol, _,_,_,_,_,_ = load_trj(str_load)
# determine when GMol is out of cloud
z_diff = - 100
k = 0
while z_diff < 0:
    z_diff = r_z_GMol[k]*1000 + dim_nu[2]
    k += 1
time_in = t_GMol[k]
z_diff = - 100
k = 0
while z_diff < 0:
    z_diff = r_z_GMol[k]*1000 - dim_nu[2]
    k += 1
time_out = t_GMol[k]
print(time_in*1000,time_out*1000)

# Focus on SimuType2
fluo_signal, temp_signal, Inje_signal, tt = load_condition(all_subdir,my_cond, myslashpos, slashcond)

figure('test SimuType4');clf()
semilogy(Inje_signal[0][my_try]*1000,Inje_signal[2][my_try],marker='+')
vlines(time_in*1000,0.5e-3,5e-1)
vlines(time_out*1000,0.5e-3,5e-1)
grid()
tight_layout()
xlabel('time [ms]')
ylabel('T [K]')

vlines(temps[2]*1000,0.5e-3,5e-1,color='r')
vlines(temps[1]*1000,0.5e-3,5e-1,color='r')

tight_layout()

# Cloud position at beginning of injection
file_name = '{}/xva{}'.format(pt_to_plot[my_try],sort(onlyfiles)[0].strip('.dat')[4:])
plot_XYZ(file_name,fig_name='XYZ_point'+str(pt_to_plot[to_plot]),fig_title='XYZ')

# Temps de traversée du nuage correspond elle à la longueur ?
v_GMol = 100 # m/s
l_nuage = dim_nu[2]*1e-3 * 2 # demi grand-axe x 2 = Longueur

t_trav = l_nuage/v_GMol
print('temps de traversée calculé',t_trav*1000000)
print('temps de traversée simulé ',(time_out - time_in)*1000000)


3.477334982600513 3.481767182914262


TypeError: list indices must be integers or slices, not list

In [222]:
# position des ions à la fin de la simu
to_plot = 0
onlyfiles = [f for f in listdir(pt_to_plot[to_plot]) if isfile(join(pt_to_plot[to_plot], f)) and "xva" in f and ".bin" in f]
print(sort(onlyfiles))
file_name = '{}/xva_{}'.format(pt_to_plot[to_plot],sort(onlyfiles)[0].strip('.bin')[4:])
plot_XYZ(file_name,fig_name='XYZ_point'+str(pt_to_plot[to_plot]),fig_title='XYZ')
print(sort(onlyfiles)[2])

['xva_SimuType0_N01024_Vrf0064_Udc0.1000D+01V_D1.0_S1.0RFG.bin'
 'xva_SimuType2_01_N01024_Vrf0064_Udc0.1000D+01V_D1.0_S1.0RFG50eV.bin'
 'xva_SimuType4_01_N01024_Vrf0064_Udc0.1000D+01V_D1.0_S1.0RFG50eV.bin']
xva_SimuType4_01_N01024_Vrf0064_Udc0.1000D+01V_D1.0_S1.0RFG50eV.bin


In [20]:
# Récupérer les données pour un point

# tt                  : temps
# PM, SNR             : signal PM temporel, rapport signal/bruit, ci.e fluo/bruitfluo
# PMvar, Tvar         : variation de fluo et température entre passage Gmol et fin de simu
# deltaEc, deltaEcRel : variation énergie GMol
# T_aux, T_CM         : température interne du nuage, température du centre de masse
# r_LC, v_LC, a_LC    : position, vitesse, accélération des ions Ca+
# r_LC_clip           : même chose mais en dégageant les ions perdus et beaucoup trop loin (hors piège)
# dim_nu              : dimensions x, y et z, nuage Ca+ basé sur r_LC_clip

myslashpos = slash_cfg[0]
slashcond = slash_cfg[1]

# determining number of elements on each repetition
num_runs = [runs[myslashpos[slashcond+1]+1:] for runs in all_subdir if list(points_and_coord.keys())[0] in runs]
num_runs = list(dict.fromkeys(num_runs))

cond_name = 'DC01_RF10'
pt_to_plot = [i for i in all_subdir if cond_name in i]
for k, address in enumerate(pt_to_plot):
    onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and not "xva" in f and ".dat" in f]
fluo_var, GMol_var, cloud_atlas, t_aux, r_LC_clip = load_T_PM_cloud_GMol(all_subdir[0],flag_plot=0)
        
shapevar = (len(pt_to_plot),len(cloud_atlas[2]))
PMvar = np.zeros(shapevar)
Tvar = np.zeros(shapevar)
SNR = np.zeros(shapevar)
tt = np.zeros(shapevar)
T_aux = np.zeros((len(pt_to_plot),len(cloud_atlas[2]),3))
T_CM = np.zeros((len(pt_to_plot),len(cloud_atlas[2]),3))
PM = np.zeros(shapevar)
        
t0 = time.clock()
print("Hello")
    
for k, address in enumerate(pt_to_plot):
    fluo_var, GMol_var, cloud_atlas, t_aux, r_LC_clip = load_T_PM_cloud_GMol(address,flag_plot=0)
    PMvar[k] = fluo_var[0] ; Tvar[k] = fluo_var[1] ; SNR[k] = fluo_var[2]
    tt[k] = cloud_atlas[0] ; T_aux[k] = cloud_atlas[1] ; T_CM[k] = cloud_atlas[2] ; PM[k] = cloud_atlas[3]

t1 = time.clock() - t0
print("Time elapsed: ", t1, 's') # CPU seconds elapsed (floating point)
print("Time elapsed: ", t1/60, 'm') # CPU seconds elapsed (floating point)

Hello
Time elapsed:  2.8138820000000004 s
Time elapsed:  0.04689803333333334 m


In [291]:
# Plot all temperatures rf relax après refroidissement

fig, ax = plt.subplots(4, 4,num='Temperature interaction Gmol Ca+',clear=True,sharex=True,sharey=True)
fig.set_size_inches(11.69,8.27)
# suptitle('Saturation 866')
to_plot = 0

for l in range(0,4):
    for k in range(0,4):
        ax[k,l].semilogy(tt[to_plot,:]*1e3,T_aux[to_plot,:,0],'+',mew=1,ms=5)
        ax[k,l].semilogy(tt[to_plot,:]*1e3,T_aux[to_plot,:,1],'+',mew=1,ms=5)
        ax[k,l].semilogy(tt[to_plot,:]*1e3,T_aux[to_plot,:,2],'+',mew=1,ms=5)
        ax[k,l].vlines(t_aux[0]*1e3,1e-4,1e-1,color='red')
        ax[k,l].vlines(t_aux[1]*1e3,1e-4,1e-1,color='red')
        ax[k,l].set_yticks([1e-4,1e-3,1e-2,1e-1,1,1e2,1e4])
        ax[k,l].grid()
    #     ax[k].set_ylim(-0.1,0.55)
        ax[k,l].xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=2))
        ax[k,l].annotate('try'+str(to_plot), xy=(0.80, 0.99), xycoords='axes fraction', # pts_concerned[to_plot]
                size=12, ha='left', va='top',
                bbox=dict(boxstyle='round', fc='w'))
        to_plot += 1
        if to_plot == 13:
            break
    if to_plot == 13:
        break
# ax[3].set_xlim(-10,4000)
# ax[k,l].set_ylim(2e-4,1e-1)
fig.text(0.56, 0.04, 'time [ms]', ha='center')
fig.text(0.04, 0.54, 'T [K]', va='center', rotation='vertical')

fig.suptitle(pt_to_plot[0][:-6],fontsize=10)

# fig.text(0.5, 0.01, 'f', ha='center')
# fig.text(0.01, 0.5, r'$| FFT(HC(t)) | / N_{sample}$', va='center', rotation='vertical')

fig.set_size_inches(11.69,8.27)
plt.tight_layout()
subplots_adjust(hspace=0.05, wspace = 0.05, bottom = 0.120, left = 0.140) # hspace=0, top=0.95, left = 0.065, bottom = 0.07
# savefig('FFT_lock_oscillo_0_11500.png')

In [742]:
# Plot all temperatures pendant injection SimuType4

my_cond = 'DC01_RF11'
fluo_signal, temp_signal, Inje_signal, tt = load_condition(all_subdir,my_cond, myslashpos, slashcond)
pt_to_plot = [i for i in all_subdir if my_cond in i]

fig, ax = plt.subplots(4, 4,num='Temperature_inje_'+my_cond,clear=True,sharex=True,sharey=True)
fig.set_size_inches(11.69,8.27)
# suptitle('Saturation 866')
my_try = 0
for l in range(0,4):
    for k in range(0,4):
        address = pt_to_plot[my_try]
        onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and not "xva" in f and ".dat" in f]
        # Cloud size
        r_LC_clip, dim_nu = cloud_size(address,onlyfiles)
        print(dim_nu)
        str_load = str(address+'/trj'+sort(onlyfiles)[2][4:]).strip('.dat')
        t_GMol,_, _, r_z_GMol, _,_,_,_,_,_ = load_trj(str_load)
        # determine when GMol is out of cloud
        z_diff = - 100
        kk = 0
        while z_diff < 0:
            z_diff = r_z_GMol[kk]*1000 + dim_nu[2]
            kk += 1
        time_in = t_GMol[kk]
        z_diff = - 100
        kk = 0
        while z_diff < 0:
            z_diff = r_z_GMol[kk]*1000 - dim_nu[2]
            kk += 1
        time_out = t_GMol[kk]
        print(time_in*1000,time_out*1000)
                
        ax[k,l].semilogy(Inje_signal[0][my_try]*1000,Inje_signal[2][my_try],marker='+')
        ax[k,l].vlines(t_aux[0]*1e3,1e-4,1e-1,color='red')
        ax[k,l].vlines(t_aux[1]*1e3,1e-4,1e-1,color='red')
        ax[k,l].vlines(time_in*1000,0.5e-3,5e-1)
        ax[k,l].vlines(time_out*1000,0.5e-3,5e-1)
        ax[k,l].set_yticks([1e-4,1e-3,1e-2,1e-1,1,1e2,1e4])
        ax[k,l].grid()
    #     ax[k].set_ylim(-0.1,0.55)
        ax[k,l].xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.02))
        if fluo_signal[1][my_try] < 1:
            ax[k,l].annotate('try'+str(my_try), xy=(0.99, 0.11), xycoords='axes fraction', # pts_concerned[to_plot]
                size=12, ha='right', va='top',
                bbox=dict(boxstyle='round', fc='red'))
        else:
            ax[k,l].annotate('try'+str(my_try), xy=(0.99, 0.11), xycoords='axes fraction', # pts_concerned[to_plot]
                size=12, ha='right', va='top',
                bbox=dict(boxstyle='round', fc='green'))

        my_try += 1
        if my_try == 13:
            break
    if my_try == 13:
        break
# ax[3].set_xlim(-10,4000)
ax[k,l].set_ylim(2e-4,0.5e0)
fig.text(0.56, 0.04, 'time [ms]', ha='center')
fig.text(0.04, 0.54, 'T [K]', va='center', rotation='vertical')

fig.suptitle(pt_to_plot[0][:-6],fontsize=10)

# fig.text(0.5, 0.01, 'f', ha='center')
# fig.text(0.01, 0.5, r'$| FFT(HC(t)) | / N_{sample}$', va='center', rotation='vertical')

fig.set_size_inches(11.69,8.27)
plt.tight_layout()
subplots_adjust(hspace=0.05, wspace = 0.05, bottom = 0.130, left = 0.160) # hspace=0, top=0.95, left = 0.065, bottom = 0.07
# savefig('FFT_lock_oscillo_0_11500.png')

/home/adrian/.local/lib/python3.6/site-packages/matplotlib/axes/_base.py:3449: UserWarning: Attempted to set non-positive ylimits for log-scale axis; invalid limits will be ignored.
  'Attempted to set non-positive ylimits for log-scale axis; '


[0.09858622949252448, 0.09943829707572988, 0.9694819429536532]
11
52
3.4694488385672377 3.4896490510485814
[0.099827403975612, 0.09985615757478039, 0.9943058345366019]
11
52
3.469448829984818 3.4896489045398904
[0.099930379334341, 0.09927360562694676, 0.9731525626295133]
11
52
3.469448930067362 3.4896488025573613
[0.09972505656293708, 0.09975338858217128, 0.9669705772468572]
12
51
3.4699415408788292 3.4891561864021425
[0.09997875929523466, 0.09997121810856234, 0.9776375721320507]
11
52
3.46944894833562 3.4896487903313584
[0.09978707615504889, 0.09979256819161461, 0.9444320933599657]
12
51
3.4699417141445443 3.4891562184123743
[0.09959932127851158, 0.09980960962073249, 0.9963410094250262]
11
52
3.4694489788614504 3.489648901936693
[0.09994364895106678, 0.0998403434710452, 0.9985261239369206]
11
52
3.46944888995006 3.4896489150110845
[0.09947915186143366, 0.09996625602939256, 0.9222175958977623]
12
51
3.469941718287935 3.489156183583467
[0.09967678629755566, 0.09875143001296215, 0.968177

In [485]:
# Histogrammes DE Gmol en fonction condition

# histogram for Delta energy
# two families : detection and non detection

to_plot = [0,1,2,3,11,12,13,14,22,23,24,25,31,32,33,34,40,41,42,43,49,50,51,52,58,59,60,61]

matplotlib.rcParams.update({'font.size': 10})

SNR_trig_threshold = 1
renorm = 0.5*m_GM/C_e

fname = 'Histogram_Delta_energy_GMol'
fig, ax = plt.subplots(4, 7,num='hist_SNR_'+my_cond,clear=True,sharex=True,sharey=True)
fig.set_size_inches(11.69,8.27)
ax = ax.T.flatten()

y_hist_detec_save     = []
y_hist_nondetec_save  = []

for k,w in enumerate(to_plot):
    
    cond_name = list(points_and_coord.keys())[w]
    pt_to_plot = [i for i in all_subdir if cond_name in i]    
    
    SNR = np.zeros(13)
    deltaEc = np.zeros(13)
    
    for m in range(13):
        address = pt_to_plot[m]
        fluo_var, GMol_var, cloud_atlas, t_aux, r_LC_clip = load_T_PM_cloud_GMol(address,flag_plot=0)
        SNR[m] = fluo_var[2]
        deltaEc[m] = GMol_var[0]
#     tt = cloud_atlas[0] ; T_aux = cloud_atlas[1] ; T_CM = cloud_atlas[2] ; PM = cloud_atlas[3]
    
    y_hist_detec     = [o*renorm*1000 for n,o in enumerate(deltaEc) if abs(SNR[n]) > SNR_trig_threshold]
    y_hist_nondetec  = [o*renorm*1000 for n,o in enumerate(deltaEc) if abs(SNR[n]) <= SNR_trig_threshold]
    y_hist_detec_save.append(y_hist_detec)
    y_hist_nondetec_save.append(y_hist_nondetec)
    
    # PMvar[k][my_selection]
    my_bins = arange(-200,10,5)
#     ax[k].set_title(r'Detection SNR>'+str(SNR_trig_threshold),color='xkcd:leaf green')
    ax[k].hist([y_hist_detec, y_hist_nondetec], bins = my_bins, color = ['xkcd:leaf green', 'red'],
                edgecolor = 'k', label = ['x1', 'x2'],
                histtype = 'barstacked')
    ax[k].set_xlabel(r'$\Delta E$ [meV]')
    ax[k].set_ylim(0,11)
    ax[k].grid()
    ax[k].xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=40))
    ax[k].yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=2))
    ax[k].annotate(cond_name+'-'+str(len(y_hist_detec))+'detec', xy=(-200,10.99), xycoords='data',
        size=10, ha='left', va='top', color='xkcd:leaf green',
        bbox=dict(boxstyle='round', fc='white'))
    
#     plt.setp(ax[k].get_xticklabels(),visible=False)

fig.set_size_inches(11.69, 8.27)
fig.tight_layout()
subplots_adjust(hspace=0.05,wspace=0.05,top=0.94)

# savefig(fname+'_D'+str(Detuning)+'_'+str(whatisthisfile)+'.png', dpi=300)

matplotlib.rcParams.update({'font.size': 25})

In [743]:
# Plot detection efficacy vs temperature after injection

to_plot = [0,1,2,3,11,12,13,14,22,23,24,25,31,32,33,34,40,41,42,43,49,50,51,52,58,59,60,61]
matplotlib.rcParams.update({'font.size': 10})

pt_to_plot = [bb for bb in all_subdir if list(points_and_coord.keys())[0] in bb]
T_Inje_End = np.zeros((len(to_plot),len(pt_to_plot),3))
SNR        = np.zeros((len(to_plot),len(pt_to_plot)))

for i,j in enumerate(to_plot):
#     my_cond = list(points_and_coord.keys())[j]        
#     fluo_var, GMol_var, cloud_atlas, t_aux, r_LC_clip = load_T_PM_cloud_GMol(all_subdir[j*13],flag_plot=0)        
    shapevar = (len(pt_to_plot),len(cloud_atlas[2]))
#     PMvar = np.zeros(shapevar)
#     Tvar  = np.zeros(shapevar)
#     SNR   = np.zeros(shapevar)
#     tt    = np.zeros(shapevar)
#     T_aux = np.zeros((len(pt_to_plot),len(cloud_atlas[2]),3))
#     T_CM  = np.zeros((len(pt_to_plot),len(cloud_atlas[2]),3))
#     PM    = np.zeros(shapevar)
    print(j)
    
    for my_try, address in enumerate(pt_to_plot):
            address = all_subdir[j*13+my_try]
            print('>',address)
            onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and not "xva" in f and ".dat" in f]
            
            # Cloud size
            r_LC_clip, dim_nu = cloud_size(address,onlyfiles)
            str_load = str(address+'/trj'+sort(onlyfiles)[2][4:]).strip('.dat')
            t_GMol,_, _, r_z_GMol, _,_,_,_,_,_ = load_trj(str_load)
            # determine when GMol is out of cloud
            z_diff = - 100
            kk = 0
            while z_diff < 0:
                z_diff = r_z_GMol[kk]*1000 - dim_nu[2]
                kk += 1
            time_out = t_GMol[kk]
            
            fluo_var, GMol_var, cloud_atlas, t_aux, r_LC_clip = load_T_PM_cloud_GMol(all_subdir[j*13+my_try],flag_plot=0)
            SNR[i,my_try] = fluo_var[2]
#             PMvar[k] = fluo_var[0] ; Tvar[k] = fluo_var[1] ; SNR[k] = fluo_var[2]
#             tt[k] = cloud_atlas[0] ; T_aux[k] = cloud_atlas[1] ; T_CM[k] = cloud_atlas[2] ; PM[k] = cloud_atlas[3]
            
            _, _, T_Inje_End_tmp, _ = plot_T_and_PM_Inje(address+'/',sort(onlyfiles)[0].strip('.dat'))            
            T_Inje_End[i,my_try] = mean(T_Inje_End_tmp[kk:kk+5,:],axis = 0)

0
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC01_RF08/Try00
[0.04455390774211819, 0.04432188049182878, 0.378570811787453]
3.483247424150285
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC01_RF08/Try01
[0.044384760420993166, 0.0447447539782235, 0.37862042860667666]
3.4832486792157056
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC01_RF08/Try02
[0.04478037651871494, 0.04444092128286025, 0.3788962822119676]
3.4832461053216006
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC01_RF08/Try03
[0.04373235800117228, 0.04328602544975139, 0.378476516048731]
3.483247670679625
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC01_RF08/Try04
[0.04448151281716495, 0.04472260021446821, 0.37813471113879055]
3.4832465298998843
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC01_RF08/Try05
[0.04373996607855758, 0.04376006839429768, 0.37813932100244085]
3.483244926146684
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131

> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC03_RF08/Try01
[0.052788324745979824, 0.05301619311769742, 0.2721624062161256]
3.482258683902006
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC03_RF08/Try02
[0.053652468733391266, 0.053794093683495275, 0.27202506866264825]
3.4822587132062313
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC03_RF08/Try03
[0.05327260746757817, 0.05245615796968783, 0.2721856319571695]
3.482261581853104
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC03_RF08/Try04
[0.05347495481795905, 0.05356247372377065, 0.2725598637857896]
3.4822604293626473
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC03_RF08/Try05
[0.05392952371084062, 0.05384158598748822, 0.26908043214886196]
3.482259836388174
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC03_RF08/Try06
[0.05304427881353071, 0.05288950018374434, 0.27150069549027905]
3.4822610638718636
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200

> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC05_RF08/Try02
[0.05983911268994036, 0.059862640094589185, 0.2177690035586174]
3.4817677706346517
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC05_RF08/Try03
[0.059797650246782845, 0.059894419414687934, 0.21879903141184343]
3.4817690645756607
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC05_RF08/Try04
[0.05996255164120275, 0.0600119216908497, 0.216939677784584]
3.481770064842136
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC05_RF08/Try05
[0.05962185083973129, 0.05994377814848814, 0.21735176559290112]
3.481768675663035
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC05_RF08/Try06
[0.05933913782389009, 0.05938132435635094, 0.217572826090984]
3.4817700981819435
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC05_RF08/Try07
[0.05990391479518769, 0.06005989733653595, 0.21766192923969846]
3.4817677346763176
> /home/adrian/Documents/Programmes/Fortran_Jofre/2020013

> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC07_RF08/Try04
[0.06550800680020573, 0.06566406267220609, 0.18563186389747152]
3.4812748418577084
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC07_RF08/Try05
[0.06489643369528142, 0.0652368592731262, 0.18579613231015232]
3.4812737100138467
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC07_RF08/Try06
[0.06512247688625418, 0.06548522248179305, 0.18577030474604933]
3.481273991814624
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC07_RF08/Try07
[0.06542363708484032, 0.06493748382281829, 0.18545351160609885]
3.4812759218817884
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC07_RF08/Try08
[0.06534450130647405, 0.06551638946126818, 0.18578655607180847]
3.481273989490517
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC07_RF08/Try09
[0.0654672296749944, 0.06499726015940432, 0.18488357807989578]
3.481275759927427
> /home/adrian/Documents/Programmes/Fortran_Jofre/2020013

> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC09_RF08/Try04
[0.06978701172911242, 0.06956767347916744, 0.1627590818817252]
3.4812743576563956
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC09_RF08/Try05
[0.07019015418201338, 0.0694845151956381, 0.16253964965191442]
3.4812756693933045
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC09_RF08/Try06
[0.06991385645852943, 0.06960393693022619, 0.1626860261294305]
3.4812774603145185
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC09_RF08/Try07
[0.06985631062502588, 0.06985808328014757, 0.16295714751784393]
3.481273914337325
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC09_RF08/Try08
[0.07035917609070488, 0.07026892243255252, 0.16246409337206494]
3.4812765287594174
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC09_RF08/Try09
[0.0701258978196672, 0.0699375445977496, 0.16302540704415017]
3.481274513695341
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/

> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC10_RF08/Try05
[0.07412464839267308, 0.07422188152035761, 0.14475086249312263]
3.4812777275174263
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC10_RF08/Try06
[0.07416619313848304, 0.07430480149536114, 0.14430616865139928]
3.4812740961742774
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC10_RF08/Try07
[0.07495526764105048, 0.07410170616837475, 0.14445686905323077]
3.4812734431143606
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC10_RF08/Try08
[0.07417717568259313, 0.0746109691165712, 0.14492402231348056]
3.4812753675018566
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC10_RF08/Try09
[0.07411787079952477, 0.07442635258997458, 0.14369136889512818]
3.481273712028545
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC10_RF08/Try10
[0.07451389159237332, 0.07436547220997185, 0.14461288729730795]
3.48127366542592
> /home/adrian/Documents/Programmes/Fortran_Jofre/202001

> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC11_RF08/Try06
[0.07870108473225819, 0.07886279240331039, 0.13024929419531547]
3.480782464790632
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC11_RF08/Try07
[0.07871426775825913, 0.07862632297738897, 0.13018181934834963]
3.4807835595556753
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC11_RF08/Try08
[0.0787773350577464, 0.07934157051983382, 0.13037418554652602]
3.4807826212908815
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC11_RF08/Try09
[0.07905404128966237, 0.07938485530102106, 0.12998649107848423]
3.480784709323532
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC11_RF08/Try10
[0.07831512997979326, 0.07850449539412209, 0.13069129652489084]
3.480781428755517
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC11_RF08/Try11
[0.07938021635555974, 0.07859949159476562, 0.130024049030253]
3.480781695149599
> /home/adrian/Documents/Programmes/Fortran_Jofre/20200131/

In [749]:
# display T Inje End in function of Detection (SNR)
# Histogramme
# two families : detection and non detection
to_plot = [0,1,2,3,11,12,13,14,22,23,24,25,31,32,33,34,40,41,42,43,49,50,51,52,58,59,60,61]
matplotlib.rcParams.update({'font.size': 10})

fname = 'Histogram_TInjeEnd_Detec'
fig, ax = plt.subplots(4, 7,num=fname,clear=True,sharex=True,sharey=True)
fig.set_size_inches(11.69,8.27)
fig.suptitle('T nuage après sortie GMol : moyenne T sur 5 éch après sortie nuage')

y_hist_detec_save     = []
y_hist_nondetec_save  = []

for k,w in enumerate(to_plot):
    i = 3 - k % 4
    j = k//4
    
    cond_name = list(points_and_coord.keys())[w]
    pt_to_plot = [i for i in all_subdir if cond_name in i]    
    
#     T_Inje_End[i,my_try]
    try:
        y_hist_detec     = [o for n,o in enumerate(sum(T_Inje_End[k],axis=1)) if abs(SNR[k,n]) > SNR_trig_threshold]
    except(AxisError):
        y_hist_detec     = [0]
    try:
        y_hist_nondetec  = [o for n,o in enumerate(sum(T_Inje_End[k],axis=1)) if abs(SNR[k,n]) <= SNR_trig_threshold]
    except(AxisError):
        y_hist_nondetec  = [0]
        
    y_hist_detec_save.append(y_hist_detec)
    y_hist_nondetec_save.append(y_hist_nondetec)
    
    # PMvar[k][my_selection]
    my_bins = arange(0,0.55,0.02)  # arange(0,2,0.2)
#     ax[k].set_title(r'Detection SNR>'+str(SNR_trig_threshold),color='xkcd:leaf green')
#     hist, bins, _ = plt.hist([y_hist_detec, y_hist_nondetec], bins=8)
#     logbins = np.logspace(np.log10(bins[0]),np.log10(bins[-1]),len(bins))
    ax[i,j].hist([y_hist_detec, y_hist_nondetec], bins = my_bins, color = ['xkcd:leaf green', 'red'],
                edgecolor = 'k', label = ['x1', 'x2'],
                histtype = 'barstacked')
    ax[i,j].set_ylim(0,11)
    ax[i,j].grid()
    ax[i,j].xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.1))
    ax[i,j].yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=2))
    ax[i,j].annotate(cond_name, xy=(0.62,0.95), xycoords='axes fraction',
        size=10, ha='left', va='top', color='xkcd:black',
        bbox=dict(boxstyle='round', fc='white'))
    
#     plt.setp(ax[k].get_xticklabels(),visible=False

ax[3,3].set_xlabel(r'T [mK]')

fig.set_size_inches(11.69, 8.27)
fig.tight_layout()
subplots_adjust(top=0.957,
bottom=0.084,
left=0.034,
right=0.979,
hspace=0.0,
wspace=0.0)

# savefig(fname+'_D'+str(Detuning)+'_'+str(whatisthisfile)+'.png', dpi=300)

matplotlib.rcParams.update({'font.size': 25})

In [696]:
y_hist_detec

[1337.6199790457038,
 1258.7540105100525,
 1071.0443714705775,
 1322.8757648051662,
 1241.203406152426,
 1295.8971632905555,
 1244.4642697608076,
 1129.6431894126488,
 1279.1343301361137,
 1250.2917282482058,
 1131.1950050904304,
 1482.8468034174869,
 1311.6704591900466]

In [656]:
for k in range(4*7):
    print(max(sum(T_Inje_End[k],axis=1)))

0.28579762455272717
0.5976715826591393
2.966030342776638
957.2671897745134
0.451274273406014
1.74696483310692
2.9052854644541424
1243.0268684081752
0.14986659254828136
0.19533197736580732
1.6742607900934225
1238.689754436141
0.32616251211933645
0.16231405973722027
0.30869898783858085
7.671803753063347
0.1698344726263906
0.26927963505674335
0.6815745185479725
3.2723810472638477
0.033876351167492
0.1394885490001428
1.0876077613029649
1.6040436926913162
0.04174182157100332
0.1262866789876147
0.8428898919952064
9.147617602024663


In [634]:
# Plot all fluo rf relax après refroidissement

fig, ax = plt.subplots(4, 4,num='Fluo interaction Gmol Ca+',clear=True,sharex=True,sharey=True)
fig.set_size_inches(11.69,8.27)
# suptitle('Saturation 866')
to_plot = 0

for l in range(0,4):
    for k in range(0,4):
        ax[k,l].plot(tt[to_plot,:]*1e3,PM[to_plot,:],'+',mew=1,ms=5)
        ax[k,l].plot(tt[to_plot,:]*1e3,PM[to_plot,:],'+',mew=1,ms=5)
        ax[k,l].plot(tt[to_plot,:]*1e3,PM[to_plot,:],'+',mew=1,ms=5)
        ax[k,l].vlines(t_aux[0]*1e3,1e-4,1e-1,color='red')
        ax[k,l].vlines(t_aux[1]*1e3,1e-4,1e-1,color='red')
        ax[k,l].set_yticks([1e-4,1e-3,1e-2,1e-1,1,1e2,1e4])
        ax[k,l].grid()
    #     ax[k].set_ylim(-0.1,0.55)
        ax[k,l].xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=2))
        ax[k,l].annotate('try'+str(to_plot), xy=(0.80, 0.99), xycoords='axes fraction', # pts_concerned[to_plot]
                size=12, ha='left', va='top',
                bbox=dict(boxstyle='round', fc='w'))
        to_plot += 1
        if to_plot == 13:
            break
    if to_plot == 13:
        break
# ax[3].set_xlim(-10,4000)
# ax[k,l].set_ylim(2e-4,1e-1)
fig.text(0.56, 0.04, 'time [ms]', ha='center')
fig.text(0.04, 0.54, 'T [K]', va='center', rotation='vertical')

fig.suptitle(pt_to_plot[0][:-6],fontsize=10)

# fig.text(0.5, 0.01, 'f', ha='center')
# fig.text(0.01, 0.5, r'$| FFT(HC(t)) | / N_{sample}$', va='center', rotation='vertical')

fig.set_size_inches(11.69,8.27)
plt.tight_layout()
subplots_adjust(hspace=0.05, wspace = 0.05, bottom = 0.120, left = 0.140) # hspace=0, top=0.95, left = 0.065, bottom = 0.07
# savefig('FFT_lock_oscillo_0_11500.png')

In [43]:
# position des ions à la fin de la simu
to_plot = 9
onlyfiles = [f for f in listdir(pt_to_plot[to_plot]) if isfile(join(pt_to_plot[to_plot], f)) and "xva" in f and ".bin" in f]
print(sort(onlyfiles))
file_name = '{}/xva_{}'.format(pt_to_plot[to_plot],sort(onlyfiles)[0].strip('.bin')[4:])
plot_XYZ(file_name,fig_name='XYZ_point'+str(pt_to_plot[to_plot]),fig_title='XYZ')
print(sort(onlyfiles)[2])

['xva_SimuType0_N01024_Vrf0064_Udc0.1000D+01V_D1.0_S1.0RFG.bin'
 'xva_SimuType2_01_N01024_Vrf0064_Udc0.1000D+01V_D1.0_S1.0RFG50eV.bin'
 'xva_SimuType4_01_N01024_Vrf0064_Udc0.1000D+01V_D1.0_S1.0RFG50eV.bin']
xva_SimuType4_01_N01024_Vrf0064_Udc0.1000D+01V_D1.0_S1.0RFG50eV.bin


In [42]:
# pos ions par rapport à potentiel axial

Udc = 2
wz_LC = 2*np.pi*90806.9982303
wz2   = Udc*wz_LC**2 # * C_e*m_Ca / (C_e*m_GM)
eta = 1. / (4.* (sqrt(10) + 5. ) )
L = 0.0140608827209
cte_gauss = -1/(2*eta*L**2)

r_LC,v_LC,a_LC = load_xyz_init_bin_DP(file_name)

r_z = linspace(-0.02,0.02,1000)
gauss_pot = wz2*eta*L**2*m_Ca * (1-exp(cte_gauss*r_z**2)) /C_e
quad_pot = 0.5*wz2*m_Ca*r_z**2 /C_e

fig = figure('axial_potential');clf()
ax0 = fig.add_subplot(111)
title(pt_to_plot[0][:-6])

ax0.plot(r_z*1e3,gauss_pot,color='C0')
ax0.plot(r_z*1e3,quad_pot,color='C1')
ax0.plot(r_LC[2],r_LC[0]-min(r_LC[0]),ls='',marker='o',color='C4')

ax0.set_xlabel('z [mm]')
ax0.set_ylabel('$E_p$ [eV]')

ax0.grid()

tight_layout()

In [56]:
z0 = 400*1e-6
print( (-3/(eta*L**2)/12)*z0**2 * 0.5*m_Ca*wz2*z0**2 / C_e )

-7.146401446612211e-05


In [ ]:
Vrf = 59.23*2
r_ion = 100*1e-6
# calculate equivalent speed in RF field for a given radius
v_ion = C_e/m_Ca/Omega*Vrf/r0**2*r_ion
print(v_ion)

v_ion = 100
# calculate equivalent radius in RF field for a given atomic speed
r_ion = m_Ca*Omega/C_e*r0**2/Vrf*v_ion
print(r_ion*1e6)